In [1]:
import os
import pandas
import pickle
import sys
sys.path.append('../libs')
import prepare_data

In [6]:
metadata = pandas.read_csv('/scr-ssd/mimic/metadata_matched.csv')
filtered = prepare_data.filter_downloaded(metadata)
filtered = filtered[filtered['sig_len'] > prepare_data.CHUNK_SIZE]
print(len(filtered), 'records')

82685 records


In [6]:
metadata = pandas.read_csv('/scr-ssd/mimic/metadata.csv')
filtered = prepare_data.filter_downloaded(metadata)
filtered = filtered[filtered['sig_len'] > prepare_data.CHUNK_SIZE]
print(len(filtered), 'records')

/sailhome/kuprel/blood-pressure/env3p7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


188594 records


### Sig Data

In [2]:
hdrs = pickle.load(open('/scr1/mimic/headers.pkl', 'rb'))

keys = ['sig_name', 'baseline', 'adc_gain', 'adc_res', 'adc_zero', 'fmt', 'init_value', 'checksum', 'units']

sig_data_raw = [
    {
        'rec_id': int(i['record_name'].split('_')[0]),
        'segment': int(i['record_name'].split('_')[1]),
        'sig_index': j,
        **{k: i[k][j] for k in keys}
    }
    for i in hdrs if '_' in i['record_name'] for j in range(i['n_sig'])
]

sig_data = pandas.DataFrame(sig_data_raw)
sig_data.set_index(['rec_id', 'segment', 'sig_index'], verify_integrity=True, inplace=True)
sig_data.sort_index(inplace=True)
sig_data.to_csv('/scr-ssd/mimic/sig_data.csv')

### Metadata

In [4]:
# hdrs = pickle.load(open('/scr1/mimic/headers.pkl', 'rb'))


metadata_raw = [
    {
        'rec_id': int(i['record_name'].split('_')[0]),
        'segment': int(i['record_name'].split('_')[1]),
        'sig_len': i['sig_len'],
        'sig_name': i['sig_name'],
    }
    for i in hdrs if '_' in i['record_name']
]

metadata = pandas.DataFrame(metadata_raw)
metadata.set_index(['rec_id', 'segment'], inplace=True, verify_integrity=True)

metadata_matched = pandas.read_csv('/scr-ssd/mimic/metadata_matched.csv')
metadata_matched.set_index(['rec_id', 'segment'], inplace=True, verify_integrity=True)
metadata_matched = metadata_matched.drop(columns=['sig_len', 'sig_name'])

metadata_merged = metadata_matched.merge(metadata, on=['rec_id', 'segment'], how='outer')
metadata_merged.sort_index(inplace=True)
metadata_merged.to_csv('/scr-ssd/mimic/metadata.csv')

KeyError: "None of ['rec_id'] are in the columns"

In [13]:
! df -h | grep nvme

/dev/nvme0n1                   1.1T 1017G   28G  98% /scr-ssd


In [5]:
metadata = pandas.read_csv('/scr-ssd/mimic/metadata.csv')

/sailhome/kuprel/blood-pressure/env3p7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
metadata.rename(columns={'rec_id_prefix': 'rec_id'}, inplace=True)

In [10]:
metadata[:10]

,rec_id,segment,hadm_id,subject_id,time,gender,dob,age,weight,height,icd_codes,sig_len,sig_name
0,3000003,1,173995.0,9950.0,2155-04-19 19:44:07.664,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",16878,"['II', 'V']"
1,3000003,2,173995.0,9950.0,2155-04-19 19:46:22.685,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",384,"['II', 'V']"
2,3000003,3,173995.0,9950.0,2155-04-19 19:46:25.757,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",1028,"['II', 'V']"
3,3000003,4,173995.0,9950.0,2155-04-19 19:46:33.981,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",384,"['II', 'V']"
4,3000003,5,173995.0,9950.0,2155-04-19 19:46:37.053,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",438826,"['II', 'V']"
5,3000003,6,173995.0,9950.0,2155-04-19 20:45:07.664,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",4826,"['II', 'V', 'ABP']"
6,3000003,7,173995.0,9950.0,2155-04-19 20:45:46.269,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",2155174,"['II', 'V', 'ABP']"
7,3000003,8,173995.0,9950.0,2155-04-19 01:33:40.276,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",217500,"['II', 'V', 'ABP']"
8,3000003,9,173995.0,9950.0,2155-04-19 03:02:40.995,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",877500,"['II', 'V', 'ABP']"
9,3000003,10,173995.0,9950.0,2155-04-19 04:59:43.913,M,2108-02-01,47.0,NaN,74.0,"['41001', '42741', '4275', '4280', '41402', '2...",1343738,"['II', 'V', 'ABP']"


In [ ]:
metadata = pandas.read_csv('/scr-ssd/mimic/metadata.csv')
metadata.rename(columns={'rec_id_prefix': 'rec_id'}, inplace=True)
for k in ['subject_id', 'hadm_id']:
    metadata.loc[metadata[k].isna(), k] = -1
    metadata[k] = metadata[k].astype('int')
metadata.set_index(['rec_id', 'segment'], inplace=True, verify_integrity=True)
metadata.sort_index(inplace=True)
metadata.to_csv('/scr-ssd/mimic/metadata.csv')

In [32]:
metadata['hadm_id'].isna().sum()

0

In [35]:
metadata['icd_codes'].notna().sum()

782187

In [39]:
metadata.sort_index(inplace=True)

In [36]:
metadata.set_index(['rec_id', 'segment'], inplace=True, verify_integrity=True)

In [40]:
metadata[:5]

hadm_id  subject_id                     time gender  \
rec_id  segment                                                        
3000003 1         173995        9950  2155-04-19 19:44:07.664      M   
        2         173995        9950  2155-04-19 19:46:22.685      M   
        3         173995        9950  2155-04-19 19:46:25.757      M   
        4         173995        9950  2155-04-19 19:46:33.981      M   
        5         173995        9950  2155-04-19 19:46:37.053      M   

                        dob   age  weight  height  \
rec_id  segment                                     
3000003 1        2108-02-01  47.0     NaN    74.0   
        2        2108-02-01  47.0     NaN    74.0   
        3        2108-02-01  47.0     NaN    74.0   
        4        2108-02-01  47.0     NaN    74.0   
        5        2108-02-01  47.0     NaN    74.0   

                                                         icd_codes  sig_len  \
rec_id  segment                                                               
3000003 1        ['41001', '42741', '4275', '4280', '41402', '2...    16878   
        2        ['41001', '42741', '4275', '4280', '41402', '2...      384   
        3        ['41001', '42741', '4275', '4280', '41402', '2...     1028   
        4        ['41001', '42741', '4275', '4280', '41402', '2...      384   
        5        ['41001', '42741', '4275', '4280', '41402', '2...   438826   

                    sig_name  
rec_id  segment               
3000003 1        ['II', 'V']  
        2        ['II', 'V']  
        3        ['II', 'V']  
        4        ['II', 'V']  
        5        ['II', 'V']

In [21]:
metadata.loc[metadata['hadm_id']==-1, 'hadm_id'][:10]

132    -1
133    -1
134    -1
135    -1
136    -1
137    -1
138    -1
139    -1
1563   -1
1564   -1
Name: hadm_id, dtype: int64